In [1]:
import requests
import pandas as pd 
import numpy as np
import re
import sys, getopt
import csv
import os
import matplotlib.pyplot as plt
import seaborn as sns

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)


In [15]:
os.chdir('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings')

os.getcwd()

'/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings'

In [3]:
latest_match_data = load_csv_dataset('Analysis_Q1/latest_match_data_june_18_SUPERSTAR_INFO_ADDED.csv')
latest_match_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)
latest_match_data['mv_difference_abs'] = latest_match_data['Market.Value.Difference'].abs()
# Assuming 'latest_match_data' is already loaded
latest_match_data['MV_Prediction_Correct'] = np.where(
    (latest_match_data['Market.Value.Difference'] < 0) & (latest_match_data['Score.Difference'] < 0), 1,
    np.where(
        (latest_match_data['Market.Value.Difference'] > 0) & (latest_match_data['Score.Difference'] > 0), 1, 0
    )
)
pd.set_option('display.max_columns', None)

# ABOVE INTERVALS

## As the more valuable team

In [ ]:
#MANUAL VERSION OF SCRIPT

i = 10000
country_name = 'France'
data_subset = latest_match_data[(latest_match_data['Team.1.Name'] == country_name) | (latest_match_data['Team.2.Name'] == country_name)]

filtered_df = data_subset[data_subset['mv_difference_abs'] >= i]
#make sure it's counting only the matches where the team in question is more valuable 
filtered_df_more_valuable = filtered_df[
    ((filtered_df['Team.1.Name'] == country_name) & (filtered_df['Market.Value.Difference'] > 0)) |
    ((filtered_df['Team.2.Name'] == country_name) & (filtered_df['Market.Value.Difference'] < 0))
]
# Calculate correct predictions directly within the DataFrame
team_mv_predictions_correct = filtered_df_more_valuable[
    ((filtered_df_more_valuable['Team.1.Name'] == country_name) & (filtered_df_more_valuable['Market.Value.Difference'] >= 0) & (filtered_df_more_valuable['MV_Prediction_Correct'] == 1)) |
    ((filtered_df_more_valuable['Team.2.Name'] == country_name) & (filtered_df_more_valuable['Market.Value.Difference'] < 0) & (filtered_df_more_valuable['MV_Prediction_Correct'] == 1))
].shape[0]

#this is telling you what percentage of the time you were correct when predicting the market value of the team in question
team_mv_predictions_correct / len(filtered_df_more_valuable)

In [19]:
def create_above_intervals_table(competition, data_subset):
        # Initialize an empty list to store the results
        results_list = []
        # Assuming 'latest_match_data' is your DataFrame and already loaded
        # Loop through intervals of 10000
        for i in range(10000, 882000000 + 10000, 10000):
            # Filter the data for the current interval
            filtered_df = data_subset[data_subset['mv_difference_abs'] >= i]
            filtered_df_more_valuable = filtered_df[
                ((filtered_df['Team.1.Name'] == country_name) & (filtered_df['Market.Value.Difference'] > 0)) |
                ((filtered_df['Team.2.Name'] == country_name) & (filtered_df['Market.Value.Difference'] < 0))
            ]
            # Calculate correct predictions directly within the DataFrame
            team_mv_predictions_correct = filtered_df_more_valuable[
                ((filtered_df_more_valuable['Team.1.Name'] == country_name) & (filtered_df_more_valuable['Market.Value.Difference'] >= 0) & (filtered_df_more_valuable['MV_Prediction_Correct'] == 1)) |
                ((filtered_df_more_valuable['Team.2.Name'] == country_name) & (filtered_df_more_valuable['Market.Value.Difference'] < 0) & (filtered_df_more_valuable['MV_Prediction_Correct'] == 1))
            ].shape[0]
            # Calculate the average
            if len(filtered_df_more_valuable) > 0:
                average_mv_prediction_correct = team_mv_predictions_correct / len(filtered_df_more_valuable)
            else:
                average_mv_prediction_correct = np.nan  # Handle division by zero if no rows match
            # Store the results in a list
            results_list.append({
                'interval': i,
                'sum_mv_prediction_correct': team_mv_predictions_correct,
                'average_mv_prediction_correct': average_mv_prediction_correct,
                'sample_size': len(filtered_df_more_valuable)
            })
        # Combine all results into a single DataFrame
        results_summary_at_below_whole_dataset = pd.DataFrame(results_list)
        results_summary_at_below_whole_dataset['Competition'] = competition
        return results_summary_at_below_whole_dataset

In [21]:
def create_above_intervals_tables_country(country_name, latest_match_data):
    
    final_country_table = pd.DataFrame()

    latest_match_data_subset_country = latest_match_data[(latest_match_data['Team.1.Name'] == country_name) | (latest_match_data['Team.2.Name'] == country_name)]
    competitions_list = latest_match_data_subset_country['Competition'].unique()

    interval_table_all_competitions = create_above_intervals_table('All', latest_match_data_subset_country)

    print(country_name, 'done with ALL comps table')
    final_country_table = pd.concat([final_country_table, interval_table_all_competitions], ignore_index=True)

    # for competition in competitions_list:
    #     print(f'working on {competition}...')
    #     data_subset_competition = latest_match_data[latest_match_data['Competition'] == competition]
    #     competition_table = create_above_intervals_table(competition, data_subset_competition)
    #     final_country_table = pd.concat([final_country_table, competition_table], ignore_index=True)

    return final_country_table


In [22]:

for country_name in latest_match_data['Team.2.Name'].unique():
    
    df1 = create_above_intervals_tables_country(country_name, latest_match_data)
    csv_file_path =  f'/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/Country-by-Country interval data/{country_name}_intervals.csv'
    df1.to_csv(csv_file_path, index=False)

Iran done with ALL comps table
Chinese Taipei done with ALL comps table
Uzbekistan done with ALL comps table
DR Congo done with ALL comps table
Madagascar done with ALL comps table
Mali done with ALL comps table
Equatorial Guinea done with ALL comps table
Morocco done with ALL comps table
Sudan done with ALL comps table
Namibia done with ALL comps table
Zimbabwe done with ALL comps table
Mexico done with ALL comps table
Panama done with ALL comps table
Costa Rica done with ALL comps table
USA done with ALL comps table
Honduras done with ALL comps table
Chile done with ALL comps table
Colombia done with ALL comps table
Argentina done with ALL comps table
Peru done with ALL comps table
Brazil done with ALL comps table
Finland done with ALL comps table
Romania done with ALL comps table
Northern Ireland done with ALL comps table
Lebanon done with ALL comps table
Myanmar done with ALL comps table
Ivory Coast done with ALL comps table
Tanzania done with ALL comps table
Zambia done with ALL c

## AS THE LESS VALUABLE TEAM 

In [130]:
def create_above_intervals_table_LESS_VALUABLE_TEAM(competition, data_subset, country_name):
        # Initialize an empty list to store the results
        results_list = []
        # Assuming 'latest_match_data' is your DataFrame and already loaded
        # Loop through intervals of 10000
        for i in range(10000, 882000000 + 10000, 10000):
            # Filter the data for the current interval
            filtered_df = data_subset[data_subset['mv_difference_abs'] >= i] #all matches above X in interval difference 
            filtered_df_less_valuable = filtered_df[
                ((filtered_df['Team.1.Name'] == country_name) & (filtered_df['Market.Value.Difference'] < 0)) |
                ((filtered_df['Team.2.Name'] == country_name) & (filtered_df['Market.Value.Difference'] > 0))
            ]
            # Calculate correct predictions directly within the DataFrame
            team_mv_predictions_correct = filtered_df_less_valuable[
                ((filtered_df_less_valuable['Team.1.Name'] == country_name) & (filtered_df_less_valuable['Market.Value.Difference'] <= 0) & (filtered_df_less_valuable['Score.Difference'] > 0)) |
                ((filtered_df_less_valuable['Team.2.Name'] == country_name) & (filtered_df_less_valuable['Market.Value.Difference'] > 0) & (filtered_df_less_valuable['Score.Difference'] < 0))
            ].shape[0]
            # Calculate the average
            if len(filtered_df_less_valuable) > 0:
                average_mv_prediction_correct = team_mv_predictions_correct / len(filtered_df_less_valuable)
            else:
                average_mv_prediction_correct = np.nan  # Handle division by zero if no rows match
            # Store the results in a list
            results_list.append({
                'interval': i,
                'sum_mv_prediction_correct': team_mv_predictions_correct,
                'average_mv_prediction_correct': average_mv_prediction_correct,
                'sample_size': len(filtered_df_less_valuable)
            })
        # Combine all results into a single DataFrame
        results_summary_at_above_whole_dataset = pd.DataFrame(results_list)
        results_summary_at_above_whole_dataset['Competition'] = competition
        return results_summary_at_above_whole_dataset

In [129]:
def create_above_intervals_tables_country_less_valuable(country_name, latest_match_data):
    
    #final_country_table = pd.DataFrame()

    latest_match_data_subset_country = latest_match_data[(latest_match_data['Team.1.Name'] == country_name) | (latest_match_data['Team.2.Name'] == country_name)]
    competitions_list = latest_match_data_subset_country['Competition'].unique()

    interval_table_all_competitions = create_above_intervals_table_LESS_VALUABLE_TEAM('All', latest_match_data_subset_country, country_name)

    print(country_name, 'done with ALL comps table')
    #final_country_table = pd.concat([final_country_table, interval_table_all_competitions], ignore_index=True)

    # for competition in competitions_list:
    #     print(f'working on {competition}...')
    #     data_subset_competition = latest_match_data[latest_match_data['Competition'] == competition]
    #     competition_table = create_above_intervals_table(competition, data_subset_competition)
    #     final_country_table = pd.concat([final_country_table, competition_table], ignore_index=True)

    return interval_table_all_competitions


In [134]:

for country_name in latest_match_data['Team.2.Name'].unique():
    df1 = create_above_intervals_tables_country_less_valuable(country_name, latest_match_data)
    csv_file_path =  f'/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/Analysis_Q1/Winpct of more valuable team by MV difference interval/Country by Country interval data/Above Intervals as the less valuable team/{country_name}_ABOVE_intervals_as_less_valuable_team.csv'
    df1.to_csv(csv_file_path, index=False)

Iran done with ALL comps table
Chinese Taipei done with ALL comps table
Uzbekistan done with ALL comps table
DR Congo done with ALL comps table
Madagascar done with ALL comps table
Mali done with ALL comps table
Equatorial Guinea done with ALL comps table
Morocco done with ALL comps table
Sudan done with ALL comps table
Namibia done with ALL comps table
Zimbabwe done with ALL comps table
Mexico done with ALL comps table
Panama done with ALL comps table
Costa Rica done with ALL comps table
USA done with ALL comps table
Honduras done with ALL comps table
Chile done with ALL comps table
Colombia done with ALL comps table
Argentina done with ALL comps table
Peru done with ALL comps table
Brazil done with ALL comps table
Finland done with ALL comps table
Romania done with ALL comps table
Northern Ireland done with ALL comps table
Lebanon done with ALL comps table
Myanmar done with ALL comps table
Ivory Coast done with ALL comps table
Tanzania done with ALL comps table
Zambia done with ALL c

In [131]:
df1 = create_above_intervals_tables_country_less_valuable('Iceland', latest_match_data)
df1.sort_values('sample_size', ascending=False)


Iceland done with ALL comps table


,interval,sum_mv_prediction_correct,average_mv_prediction_correct,sample_size,Competition
0,10000,14,0.245614,57,All
11,120000,14,0.245614,57,All
1,20000,14,0.245614,57,All
19,200000,14,0.245614,57,All
18,190000,14,0.245614,57,All
...,...,...,...,...,...
74021,740220000,0,NaN,0,All
74020,740210000,0,NaN,0,All
74019,740200000,0,NaN,0,All
74018,740190000,0,NaN,0,All


# Below Intervals

## as the more valuable team

In [94]:
def create_below_intervals_table(competition, data_subset, country_name):
        # Initialize an empty list to store the results
        results_list = []
        # Assuming 'latest_match_data' is your DataFrame and already loaded
        # Loop through intervals of 10000
        for i in range(10000, 882000000 + 10000, 10000):
            # Filter the data for the current interval
            filtered_df = data_subset[data_subset['mv_difference_abs'] <= i]
            filtered_df_more_valuable = filtered_df[
                ((filtered_df['Team.1.Name'] == country_name) & (filtered_df['Market.Value.Difference'] > 0)) |
                ((filtered_df['Team.2.Name'] == country_name) & (filtered_df['Market.Value.Difference'] < 0))
            ]
            # Calculate correct predictions directly within the DataFrame
            team_mv_predictions_correct = filtered_df_more_valuable[
                ((filtered_df_more_valuable['Team.1.Name'] == country_name) & (filtered_df_more_valuable['Market.Value.Difference'] >= 0) & (filtered_df_more_valuable['MV_Prediction_Correct'] == 1)) |
                ((filtered_df_more_valuable['Team.2.Name'] == country_name) & (filtered_df_more_valuable['Market.Value.Difference'] < 0) & (filtered_df_more_valuable['MV_Prediction_Correct'] == 1))
            ].shape[0]
            # Calculate the average
            if len(filtered_df_more_valuable) > 0:
                average_mv_prediction_correct = team_mv_predictions_correct / len(filtered_df_more_valuable)
            else:
                average_mv_prediction_correct = np.nan  # Handle division by zero if no rows match
            # Store the results in a list
            results_list.append({
                'interval': i,
                'sum_mv_prediction_correct': team_mv_predictions_correct,
                'average_mv_prediction_correct': average_mv_prediction_correct,
                'sample_size': len(filtered_df_more_valuable)
            })
        # Combine all results into a single DataFrame
        results_summary_at_below_whole_dataset = pd.DataFrame(results_list)
        results_summary_at_below_whole_dataset['Competition'] = competition
        return results_summary_at_below_whole_dataset

In [95]:
def create_below_intervals_tables_country(country_name, latest_match_data):
    
    final_country_table = pd.DataFrame()

    latest_match_data_subset_country = latest_match_data[(latest_match_data['Team.1.Name'] == country_name) | (latest_match_data['Team.2.Name'] == country_name)]
    competitions_list = latest_match_data_subset_country['Competition'].unique()

    interval_table_all_competitions = create_below_intervals_table('All', latest_match_data_subset_country, country_name)

    print(country_name, 'done with ALL comps table')
    final_country_table = pd.concat([final_country_table, interval_table_all_competitions], ignore_index=True)

    # for competition in competitions_list:
    #     print(f'working on {competition}...')
    #     data_subset_competition = latest_match_data[latest_match_data['Competition'] == competition]
    #     competition_table = create_above_intervals_table(competition, data_subset_competition)
    #     final_country_table = pd.concat([final_country_table, competition_table], ignore_index=True)

    return final_country_table


In [106]:
for country_name in latest_match_data['Team.2.Name'].unique():
    if country_name == 'England':
        0==0
    elif country_name == 'Iran':
        0==0
    else:
        df1 = create_below_intervals_tables_country(country_name, latest_match_data)
        csv_file_path =  f'/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/Analysis_Q1/Winpct of more valuable team by MV difference interval/Country by Country interval data/Below Intervals as the more valuable team/{country_name}_intervals_below.csv'
        df1.to_csv(csv_file_path, index=False)

Chinese Taipei done with ALL comps table
Uzbekistan done with ALL comps table
DR Congo done with ALL comps table
Madagascar done with ALL comps table
Mali done with ALL comps table
Equatorial Guinea done with ALL comps table
Morocco done with ALL comps table
Sudan done with ALL comps table
Namibia done with ALL comps table
Zimbabwe done with ALL comps table
Mexico done with ALL comps table
Panama done with ALL comps table
Costa Rica done with ALL comps table
USA done with ALL comps table
Honduras done with ALL comps table
Chile done with ALL comps table
Colombia done with ALL comps table
Argentina done with ALL comps table
Peru done with ALL comps table
Brazil done with ALL comps table
Finland done with ALL comps table
Romania done with ALL comps table
Northern Ireland done with ALL comps table
Lebanon done with ALL comps table
Myanmar done with ALL comps table
Ivory Coast done with ALL comps table
Tanzania done with ALL comps table
Zambia done with ALL comps table
Lesotho done with AL

## as the less valuable team

In [136]:
def create_below_intervals_table_less_valuable(competition, data_subset, country_name):
        # Initialize an empty list to store the results
        results_list = []
        # Assuming 'latest_match_data' is your DataFrame and already loaded
        # Loop through intervals of 10000
        for i in range(10000, 882000000 + 10000, 10000):
            # Filter the data for the current interval
            filtered_df = data_subset[data_subset['mv_difference_abs'] <= i]
            filtered_df_less_valuable = filtered_df[
                ((filtered_df['Team.1.Name'] == country_name) & (filtered_df['Market.Value.Difference'] < 0)) |
                ((filtered_df['Team.2.Name'] == country_name) & (filtered_df['Market.Value.Difference'] > 0))
            ]
            # Calculate correct predictions directly within the DataFrame
            team_mv_predictions_correct = filtered_df_less_valuable[
                ((filtered_df_less_valuable['Team.1.Name'] == country_name) & (filtered_df_less_valuable['Market.Value.Difference'] <= 0) & (filtered_df_less_valuable['Score.Difference'] > 0)) |
                ((filtered_df_less_valuable['Team.2.Name'] == country_name) & (filtered_df_less_valuable['Market.Value.Difference'] > 0) & (filtered_df_less_valuable['Score.Difference'] < 0))
            ].shape[0]
            #you want all the matches where they were less valuable and they won as being less valuable. 


            # Calculate the average
            if len(filtered_df_less_valuable) > 0:
                average_mv_prediction_correct = team_mv_predictions_correct / len(filtered_df_less_valuable)
            else:
                average_mv_prediction_correct = np.nan  # Handle division by zero if no rows match
            # Store the results in a list
            results_list.append({
                'interval': i,
                'sum_mv_prediction_correct': team_mv_predictions_correct,
                'average_mv_prediction_correct': average_mv_prediction_correct,
                'sample_size': len(filtered_df_less_valuable)
            })
        # Combine all results into a single DataFrame
        results_summary_at_below_whole_dataset = pd.DataFrame(results_list)
        results_summary_at_below_whole_dataset['Competition'] = competition
        return results_summary_at_below_whole_dataset

In [138]:
def create_below_intervals_tables_country_less_valuable(country_name, latest_match_data):
    
    final_country_table = pd.DataFrame()

    latest_match_data_subset_country = latest_match_data[(latest_match_data['Team.1.Name'] == country_name) | (latest_match_data['Team.2.Name'] == country_name)]
    #competitions_list = latest_match_data_subset_country['Competition'].unique()

    interval_table_all_competitions = create_below_intervals_table_less_valuable('All', latest_match_data_subset_country, country_name)

    print(country_name, 'done with ALL comps table')
    final_country_table = pd.concat([final_country_table, interval_table_all_competitions], ignore_index=True)

    # for competition in competitions_list:
    #     print(f'working on {competition}...')
    #     data_subset_competition = latest_match_data[latest_match_data['Competition'] == competition]
    #     competition_table = create_above_intervals_table(competition, data_subset_competition)
    #     final_country_table = pd.concat([final_country_table, competition_table], ignore_index=True)

    return final_country_table


In [139]:
for country_name in latest_match_data['Team.2.Name'].unique():
    df1 = create_below_intervals_tables_country_less_valuable(country_name, latest_match_data)
    csv_file_path =  f'/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/Analysis_Q1/Winpct of more valuable team by MV difference interval/Country by Country interval data/Below Intervals as the less valuable team/{country_name}_intervals_below.csv'
    df1.to_csv(csv_file_path, index=False)

Iran done with ALL comps table
Chinese Taipei done with ALL comps table
Uzbekistan done with ALL comps table
DR Congo done with ALL comps table
Madagascar done with ALL comps table
Mali done with ALL comps table
Equatorial Guinea done with ALL comps table
Morocco done with ALL comps table
Sudan done with ALL comps table
Namibia done with ALL comps table
Zimbabwe done with ALL comps table
Mexico done with ALL comps table
Panama done with ALL comps table
Costa Rica done with ALL comps table
USA done with ALL comps table
Honduras done with ALL comps table
Chile done with ALL comps table
Colombia done with ALL comps table
Argentina done with ALL comps table
Peru done with ALL comps table
Brazil done with ALL comps table
Finland done with ALL comps table
Romania done with ALL comps table
Northern Ireland done with ALL comps table
Lebanon done with ALL comps table
Myanmar done with ALL comps table
Ivory Coast done with ALL comps table
Tanzania done with ALL comps table
Zambia done with ALL c

In [135]:
data = create_below_intervals_tables_country_less_valuable('Germany', latest_match_data)
data

Germany done with ALL comps table


,interval,sum_mv_prediction_correct,average_mv_prediction_correct,sample_size,Competition
0,10000,0,NaN,0,All
1,20000,0,NaN,0,All
2,30000,0,NaN,0,All
3,40000,0,NaN,0,All
4,50000,0,NaN,0,All
...,...,...,...,...,...
88195,881960000,1,0.090909,11,All
88196,881970000,1,0.090909,11,All
88197,881980000,1,0.090909,11,All
88198,881990000,1,0.090909,11,All
